In [26]:
%matplotlib inline

In [27]:
# Dependencies

# %matplotlib notebook
# %matplotlib notebook

import os
import json
import requests
import matplotlib.pyplot as plt

import numpy as np # linear algebra
import random
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import operator
from pprint import pprint
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image

## Initial Dataset from Kaggle
TMDB 5000 Movie Dataset

In [28]:
# Original Dataframe used for Revenue, Budget information
path = "tmdb-5000-movie-dataset/tmdb_5000_movies.csv"
df = pd.read_csv(path)


## API Call
The API call to create the dataset

In [29]:
# title_list_0_5000 = list(df['title'][0:])
# print(title_list_0_5000)


# Test for one movie

# apikey = "trilogy"
# movie = "jack reacher"
# query_url = f"http://www.omdbapi.com/?apikey={apikey}&"
# response = requests.get(f'{query_url}t={movie}').json()
# pprint(response)

# set up lists to hold reponse info

# titles = []
# year = []
# rated = []
# released = []
# runtime = []
# genre = []
# director = []
# writer = []
# language = []
# country = []
# awards = []
# m_ratings = []
# i_ratings = []
# boxoffice = []
# production = []

# record = 0
# loop_set = 1


# # 

# print("Beginning Data Retrieval")
# print("------------------------")

# for title in title_list_0_5000:
#     response = requests.get(f'{query_url}t={title}').json()
#     record += 1
#     #pprint(response)
    
#     try:
#         titles.append(response['Title'])
#         year.append(response['Year'])
#         rated.append(response['Rated'])
#         released.append(response['Released'])
#         runtime.append(response['Runtime'])
#         genre.append(response['Genre'])
#         director.append(response['Director'])
#         writer.append(response['Writer'])
#         language.append(response['Language'])
#         country.append(response['Country'])
#         awards.append(response['Awards'])
#         m_ratings.append(response['Metascore'])
#         i_ratings.append(response['imdbRating'])
#         boxoffice.append(response['BoxOffice'])
#         production.append(response['Production'])
        
        
#         print(f"Processing Record {record} of Set {loop_set}: {title}")
    
#     except(KeyError):
#         print("Title is not found... skipping.")
#         loop_set += 1

# print("------------------------")
# print("Data Retrieval Complete!")
# print("------------------------")

## Post-API Dataset
Create a new data set using the OMDB API call

In [30]:
# Post-API New Dataframe used for Analysis
path = "moviesapi.csv"
new_df = pd.read_csv(path)
new_df.head()

,Title,Year,Rated,Runtime,Genre,Director,Writer,Language,Country,Awards,Metascore,IMDB Rating
0,Avatar,2009,PG-13,162 min,"Action, Adventure, Fantasy, Sci-Fi",James Cameron,James Cameron,"English, Spanish","UK, USA",Won 3 Oscars. Another 85 wins & 128 nominations.,83.0,7.8
1,Pirates of the Caribbean: At World's End,2007,PG-13,169 min,"Action, Adventure, Fantasy",Gore Verbinski,"Ted Elliott, Terry Rossio, Ted Elliott (charac...","Mandarin, English",USA,Nominated for 2 Oscars. Another 20 wins & 44 n...,50.0,7.1
2,Spectre,2015,PG-13,148 min,"Action, Adventure, Thriller",Sam Mendes,"John Logan (screenplay by), Neal Purvis (scree...","English, Spanish, Italian, German, French","UK, USA",Won 1 Oscar. Another 7 wins & 32 nominations.,60.0,6.8
3,The Dark Knight Rises,2012,PG-13,164 min,"Action, Thriller",Christopher Nolan,"Jonathan Nolan (screenplay), Christopher Nolan...","English, Arabic","UK, USA",Nominated for 1 BAFTA Film Award. Another 38 w...,78.0,8.4
4,John Carter,2012,PG-13,132 min,"Action, Adventure, Sci-Fi",Andrew Stanton,"Andrew Stanton (screenplay by), Mark Andrews (...",English,USA,2 wins & 8 nominations.,51.0,6.6


In [31]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4719 entries, 0 to 4718
Data columns (total 12 columns):
Title          4719 non-null object
Year           4719 non-null object
Rated          4604 non-null object
Runtime        4702 non-null object
Genre          4717 non-null object
Director       4696 non-null object
Writer         4652 non-null object
Language       4709 non-null object
Country        4719 non-null object
Awards         4035 non-null object
Metascore      3960 non-null float64
IMDB Rating    4708 non-null float64
dtypes: float64(2), object(10)
memory usage: 442.5+ KB


## Revenue/Budget Dataframe
OMDB API does not include Revenue and Budget, we used the Revenue and Budget data from the original dataset

In [32]:
# Highest revenue generating movies 
revenue_df = pd.DataFrame({'Title' : df['title'],
                           'Revenue': df['revenue'],
                           'Budget' : df['budget']})
revenue_df.sort_values(by= 'Revenue', ascending=False).head()

,Title,Revenue,Budget
0,Avatar,2787965087,237000000
25,Titanic,1845034188,200000000
16,The Avengers,1519557910,220000000
28,Jurassic World,1513528810,150000000
44,Furious 7,1506249360,190000000


## Final Dataframe
Final dataframe (pre-cleanup)

In [33]:
final_df = pd.merge(new_df, revenue_df, on= "Title")
final_df.head(1)

,Title,Year,Rated,Runtime,Genre,Director,Writer,Language,Country,Awards,Metascore,IMDB Rating,Revenue,Budget
0,Avatar,2009,PG-13,162 min,"Action, Adventure, Fantasy, Sci-Fi",James Cameron,James Cameron,"English, Spanish","UK, USA",Won 3 Oscars. Another 85 wins & 128 nominations.,83.0,7.8,2787965087,237000000


## Data clean-up

In [34]:
final_df['Year'].sort_values(ascending= False).head(10)

3892        2019–
4302         2019
4566         2018
4397         2018
4233         2018
4571         2018
1456    2017–2019
3995         2017
1906         2017
4457         2017
Name: Year, dtype: object

In [35]:
# Finding Year ranges to be removed from dataset

for row, item in enumerate(final_df['Year']):
   try:
      int(item)
   except: # ValueError can be defined to be more specific
      print('ERROR at index {}: {!r}'.format(row, item))

ERROR at index 1456: '2017–2019'
ERROR at index 1670: '1978–1979'
ERROR at index 1729: '1970–'
ERROR at index 2273: '1967–1969'
ERROR at index 3892: '2019–'
ERROR at index 4371: '2008–'
ERROR at index 4478: '2012–'


In [36]:
# Removing year ranges from dataset

final_df = final_df.drop(final_df.index[[1456, 1670, 1729, 2273, 3892, 4371, 4478]])

final_df['Year'].sort_values(ascending= False).head()

4302    2019
4233    2018
4566    2018
4571    2018
4397    2018
Name: Year, dtype: object

In [37]:
# Convert Year column to integer

final_df['Year'] = final_df['Year'].astype(int)

In [38]:
final_df.describe()

,Year,Metascore,IMDB Rating,Revenue,Budget
count,4582.000000,3886.000000,4576.000000,4.582000e+03,4.582000e+03
mean,2002.058708,55.112970,6.425896,8.438551e+07,2.962052e+07
std,12.996240,18.390274,1.091051,1.649530e+08,4.107635e+07
min,1909.000000,1.000000,1.200000,0.000000e+00,0.000000e+00
25%,1999.000000,42.000000,5.800000,0.000000e+00,1.000000e+06
50%,2005.000000,55.000000,6.500000,2.030899e+07,1.500000e+07
75%,2010.000000,68.000000,7.200000,9.570559e+07,4.000000e+07
max,2019.000000,100.000000,9.300000,2.787965e+09,3.800000e+08


In [39]:
# Obtaining info on the columns to see where data needs to be dropped or filled

final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4582 entries, 0 to 4588
Data columns (total 14 columns):
Title          4582 non-null object
Year           4582 non-null int64
Rated          4497 non-null object
Runtime        4571 non-null object
Genre          4582 non-null object
Director       4576 non-null object
Writer         4538 non-null object
Language       4575 non-null object
Country        4582 non-null object
Awards         3948 non-null object
Metascore      3886 non-null float64
IMDB Rating    4576 non-null float64
Revenue        4582 non-null int64
Budget         4582 non-null int64
dtypes: float64(2), int64(3), object(9)
memory usage: 537.0+ KB


In [40]:
final_df = final_df.drop(['Metascore'], axis=1)
final_df

,Title,Year,Rated,Runtime,Genre,Director,Writer,Language,Country,Awards,IMDB Rating,Revenue,Budget
0,Avatar,2009,PG-13,162 min,"Action, Adventure, Fantasy, Sci-Fi",James Cameron,James Cameron,"English, Spanish","UK, USA",Won 3 Oscars. Another 85 wins & 128 nominations.,7.8,2787965087,237000000
1,Pirates of the Caribbean: At World's End,2007,PG-13,169 min,"Action, Adventure, Fantasy",Gore Verbinski,"Ted Elliott, Terry Rossio, Ted Elliott (charac...","Mandarin, English",USA,Nominated for 2 Oscars. Another 20 wins & 44 n...,7.1,961000000,300000000
2,Spectre,2015,PG-13,148 min,"Action, Adventure, Thriller",Sam Mendes,"John Logan (screenplay by), Neal Purvis (scree...","English, Spanish, Italian, German, French","UK, USA",Won 1 Oscar. Another 7 wins & 32 nominations.,6.8,880674609,245000000
3,The Dark Knight Rises,2012,PG-13,164 min,"Action, Thriller",Christopher Nolan,"Jonathan Nolan (screenplay), Christopher Nolan...","English, Arabic","UK, USA",Nominated for 1 BAFTA Film Award. Another 38 w...,8.4,1084939099,250000000
4,John Carter,2012,PG-13,132 min,"Action, Adventure, Sci-Fi",Andrew Stanton,"Andrew Stanton (screenplay by), Mark Andrews (...",English,USA,2 wins & 8 nominations.,6.6,284139100,260000000
5,Spider-Man 3,2007,PG-13,139 min,"Action, Adventure, Sci-Fi",Sam Raimi,"Sam Raimi (screenplay), Ivan Raimi (screenplay...","English, French",USA,Nominated for 1 BAFTA Film Award. Another 3 wi...,6.2,890871626,258000000
6,Tangled,2010,PG,100 min,"Animation, Adventure, Comedy, Family, Fantasy,...","Nathan Greno, Byron Howard","Dan Fogelman (screenplay by), Jacob Grimm (bas...",English,USA,Nominated for 1 Oscar. Another 9 wins & 40 nom...,7.8,591794936,260000000
7,Avengers: Age of Ultron,2015,PG-13,141 min,"Action, Adventure, Sci-Fi",Joss Whedon,"Joss Whedon, Stan Lee (based on the Marvel com...","English, Korean",USA,7 wins & 45 nominations.,7.3,1405403694,280000000
8,Harry Potter and the Half-Blood Prince,2009,PG,153 min,"Adventure, Family, Fantasy, Mystery",David Yates,"Steve Kloves (screenplay), J.K. Rowling (novel)",English,"UK, USA",Nominated for 1 Oscar. Another 8 wins & 35 nom...,7.6,933959197,250000000
9,Batman v Superman: Dawn of Justice,2016,PG-13,151 min,"Action, Adventure, Fantasy, Sci-Fi",Zack Snyder,"Chris Terrio, David S. Goyer, Bob Kane (Batman...",English,USA,14 wins & 30 nominations.,6.5,873260194,250000000


In [41]:
# Clean runtime column

# Fill missing values with 0
final_df[['Runtime', 'IMDB Rating']] = final_df[['Runtime', 'IMDB Rating']].fillna(0)

# Convert dtype to int
final_df['Runtime'] = final_df['Runtime'].str.replace('min', '')
final_df['Runtime'] = final_df['Runtime'].replace(np.nan,0)
final_df['Runtime'] = final_df['Runtime'].astype(int)

In [42]:
# Fill missing values with "NA"
final_df[['Rated', 'Director', 'Genre', 'Writer', 'Language', 'Awards']] = final_df[['Rated', 'Director', 'Genre', 'Writer', 'Language', 'Awards']].fillna("NA")

In [43]:
# Cleaned Dataframe

final_df.to_csv("moviesapi_clean.csv", header= True, index= False)
final_df

,Title,Year,Rated,Runtime,Genre,Director,Writer,Language,Country,Awards,IMDB Rating,Revenue,Budget
0,Avatar,2009,PG-13,162,"Action, Adventure, Fantasy, Sci-Fi",James Cameron,James Cameron,"English, Spanish","UK, USA",Won 3 Oscars. Another 85 wins & 128 nominations.,7.8,2787965087,237000000
1,Pirates of the Caribbean: At World's End,2007,PG-13,169,"Action, Adventure, Fantasy",Gore Verbinski,"Ted Elliott, Terry Rossio, Ted Elliott (charac...","Mandarin, English",USA,Nominated for 2 Oscars. Another 20 wins & 44 n...,7.1,961000000,300000000
2,Spectre,2015,PG-13,148,"Action, Adventure, Thriller",Sam Mendes,"John Logan (screenplay by), Neal Purvis (scree...","English, Spanish, Italian, German, French","UK, USA",Won 1 Oscar. Another 7 wins & 32 nominations.,6.8,880674609,245000000
3,The Dark Knight Rises,2012,PG-13,164,"Action, Thriller",Christopher Nolan,"Jonathan Nolan (screenplay), Christopher Nolan...","English, Arabic","UK, USA",Nominated for 1 BAFTA Film Award. Another 38 w...,8.4,1084939099,250000000
4,John Carter,2012,PG-13,132,"Action, Adventure, Sci-Fi",Andrew Stanton,"Andrew Stanton (screenplay by), Mark Andrews (...",English,USA,2 wins & 8 nominations.,6.6,284139100,260000000
5,Spider-Man 3,2007,PG-13,139,"Action, Adventure, Sci-Fi",Sam Raimi,"Sam Raimi (screenplay), Ivan Raimi (screenplay...","English, French",USA,Nominated for 1 BAFTA Film Award. Another 3 wi...,6.2,890871626,258000000
6,Tangled,2010,PG,100,"Animation, Adventure, Comedy, Family, Fantasy,...","Nathan Greno, Byron Howard","Dan Fogelman (screenplay by), Jacob Grimm (bas...",English,USA,Nominated for 1 Oscar. Another 9 wins & 40 nom...,7.8,591794936,260000000
7,Avengers: Age of Ultron,2015,PG-13,141,"Action, Adventure, Sci-Fi",Joss Whedon,"Joss Whedon, Stan Lee (based on the Marvel com...","English, Korean",USA,7 wins & 45 nominations.,7.3,1405403694,280000000
8,Harry Potter and the Half-Blood Prince,2009,PG,153,"Adventure, Family, Fantasy, Mystery",David Yates,"Steve Kloves (screenplay), J.K. Rowling (novel)",English,"UK, USA",Nominated for 1 Oscar. Another 8 wins & 35 nom...,7.6,933959197,250000000
9,Batman v Superman: Dawn of Justice,2016,PG-13,151,"Action, Adventure, Fantasy, Sci-Fi",Zack Snyder,"Chris Terrio, David S. Goyer, Bob Kane (Batman...",English,USA,14 wins & 30 nominations.,6.5,873260194,250000000


In [44]:
print(final_df.info())
print(final_df.describe())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4582 entries, 0 to 4588
Data columns (total 13 columns):
Title          4582 non-null object
Year           4582 non-null int64
Rated          4582 non-null object
Runtime        4582 non-null int64
Genre          4582 non-null object
Director       4582 non-null object
Writer         4582 non-null object
Language       4582 non-null object
Country        4582 non-null object
Awards         4582 non-null object
IMDB Rating    4582 non-null float64
Revenue        4582 non-null int64
Budget         4582 non-null int64
dtypes: float64(1), int64(4), object(8)
memory usage: 501.2+ KB
None
              Year      Runtime  IMDB Rating       Revenue        Budget
count  4582.000000  4582.000000  4582.000000  4.582000e+03  4.582000e+03
mean   2002.058708   107.330642     6.417481  8.438551e+07  2.962052e+07
std      12.996240    21.518008     1.114829  1.649530e+08  4.107635e+07
min    1909.000000     0.000000     0.000000  0.000000e+00  0.00000